In [6]:
from vnstock3 import Vnstock

### Quote

In [2]:
from vnstock3 import Vnstock
stock = Vnstock().stock(symbol='ACB', source='VCI')

2024-09-08 22:36:40,519 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


In [3]:
stock.quote.intraday(page_size=10000, last_time='2024-09-07 14:29:52')['match_type'].value_counts()

match_type
Buy        1003
Sell        840
ATO/ATC       2
Name: count, dtype: int64

In [4]:
df = stock.quote.intraday(page_size=10000, last_time='2024-09-07 14:29:52')

In [49]:
df = stock.quote.history(start='2024-01-01', end='2024-09-06', interval='1D')

In [50]:
df

,time,open,high,low,close,volume
0,2024-01-02,20.12,20.79,20.12,20.54,13896933
1,2024-01-03,20.58,21.00,20.37,21.00,9817807
2,2024-01-04,21.17,21.55,21.08,21.25,23605373
3,2024-01-05,21.25,21.38,21.04,21.38,9282598
4,2024-01-08,21.59,21.63,21.17,21.29,12398885
...,...,...,...,...,...,...
163,2024-08-29,24.55,24.90,24.55,24.65,8588000
164,2024-08-30,24.65,24.80,24.65,24.80,4483800
165,2024-09-04,24.50,24.70,24.45,24.65,5632900
166,2024-09-05,24.65,24.70,24.45,24.50,3534700


In [15]:
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent
from langchain_openai import ChatOpenAI
import pandas as pd
from langchain_openai import OpenAI
from langchain.agents.agent_types import AgentType
from langchain_community.callbacks import get_openai_callback

In [60]:
df = stock.finance.income_statement(period='year', lang='en')
df

,ticker,yearReport,Dividends received,Dividends received,Provision for credit losses,Provision for credit losses,Revenue YoY (%),Revenue (Bn. VND),Attribute to parent company (Bn. VND),Attribute to parent company YoY (%),General & Admin Expenses,General & Admin Expenses,Interest and Similar Income,Interest and Similar Expenses,Net Interest Income,Fees and Comission Income,Fees and Comission Expenses,Net Fee and Commission Income,Net gain (loss) from foreign currency and gold dealings,Net gain (loss) from trading of trading securities,Net gain (loss) from disposal of investment securities,Net Other income/(expenses),Other expenses,Net Other income/expenses,Total operating revenue,Operating Profit before Provision,Profit before tax,Tax For the Year,Business income tax - current,Business income tax - deferred,Net Profit For the Year,Attributable to parent company,EPS_basis
0,ACB,2023,75610000000,75610000000,-1804231000000,-1804231000000,0.286199,52346796000000,16044733000000,0.172159,-10874286000000,-10874286000000,52346796000000,-27387226000000,24959570000000,4945751000000,-2023414000000,2922337000000,1110105000000,168383000000,2647140000000,1197217000000,-333846000000,863371000000,32746516000000,21872230000000,20067999000000,-4023266000000,-3993022000000,-30244000000,16044733000000,16044733000000,4092
1,ACB,2022,59456000000,59456000000,-70814000000,-70814000000,0.207186,40698834000000,13688193000000,0.425446,-11605045000000,-11605045000000,40698834000000,-17165305000000,23533529000000,5057559000000,-1531315000000,3526244000000,1048369000000,-387873000000,20648000000,1110494000000,-120800000000,989694000000,28790067000000,17185022000000,17114208000000,-3426015000000,-3445611000000,19596000000,13688193000000,13688193000000,3486
2,ACB,2021,20212000000,20212000000,-3336109000000,-3336109000000,0.058327,33713810000000,9602746000000,0.249898,-8229776000000,-8229776000000,33713810000000,-14769018000000,18944792000000,4026460000000,-1132497000000,2893963000000,871556000000,449960000000,244051000000,703159000000,-563696000000,139463000000,23563997000000,15334221000000,11998112000000,-2395366000000,-2389079000000,-6287000000,9602746000000,9602746000000,2799
3,ACB,2020,19077000000,19077000000,-941153000000,-941153000000,0.124948,31855748000000,7682823000000,0.278353,-7624268000000,-7624268000000,31855748000000,-17273625000000,14582123000000,2803484000000,-1108830000000,1694654000000,687187000000,166503000000,732115000000,495459000000,-215809000000,279650000000,18161309000000,10537041000000,9595888000000,-1913065000000,-1940681000000,27616000000,7682823000000,7682823000000,2808
4,ACB,2019,28708000000,28708000000,-273821000000,-273821000000,0.179142,28317522000000,6009937000000,0.169919,-8307596000000,-8307596000000,28317522000000,-16205362000000,12112160000000,2708759000000,-812267000000,1896492000000,430325000000,75206000000,54322000000,1546194000000,-46054000000,1500140000000,16097353000000,7789757000000,7515936000000,-1505999000000,-1463537000000,-42462000000,6009937000000,6009937000000,2780
5,ACB,2018,26387000000,26387000000,-932411000000,-932411000000,0.181879,24015362000000,5137052000000,1.425276,-6712217000000,-6712217000000,24015362000000,-13652442000000,10362920000000,2094599000000,-597073000000,1497526000000,241390000000,-78329000000,168534000000,1952302000000,-137531000000,1814771000000,14033199000000,7320982000000,6388571000000,-1251519000000,-1299419000000,47900000000,5137052000000,5137052000000,3076
6,ACB,2017,36069000000,36069000000,-2565343000000,-2565343000000,0.235368,20319639000000,2118131000000,0.598380,-6217359000000,-6217359000000,20319639000000,-11861885000000,8457754000000,1574668000000,-386337000000,1188331000000,236729000000,25305000000,603079000000,952439000000,-60797000000,891642000000,11438909000000,5221550000000,2656207000000,-538076000000,-556141000000,18065000000,2118131000000,2118131000000,1578
7,ACB,2016,24811000000,24811000000,-1217587000000,-1217587000000,0.168051,16448249000000,1325174000000,0.288789,-4

In [67]:
from vnstock3 import Vnstock
company = Vnstock().stock(symbol='FPT', source='TCBS').company
df = company.shareholders()
df

,share_holder,share_own_percent
0,Trương Gia Bình,0.0699
1,Tổng Công ty Đầu Tư Và Kinh Doanh Vốn Nhà Nước...,0.0575
2,Bùi Quang Ngọc,0.0164
3,Trương Thị Thanh Thanh,0.0149
4,Hoàng Minh Châu,0.0141
5,Phan Ngô Tống Hưng,0.0134
6,Đỗ Cao Bảo,0.0095
7,Nguyễn Thành Nam,0.0094
8,Lê Quang Tiến,0.0063
9,Khác,0.2556


In [74]:

agent = create_pandas_dataframe_agent(
    ChatOpenAI(temperature=0.1, model="gpt-4o-mini"),
    df,
    verbose=True,
    agent_type=AgentType.OPENAI_FUNCTIONS,
    allow_dangerous_code=True
)

with get_openai_callback() as cb:
    agent.agent.stream_runnable = False
    response = agent.run("3 cá nhân nào là cổ đông lớn nhất? Chiếm bao nhiêu?")
    print(cb)



> Entering new AgentExecutor chain...

Invoking: `python_repl_ast` with `{'query': "# Sorting the dataframe by share_own_percent in descending order to find the top 3 shareholders\nlargest_shareholders = df.sort_values(by='share_own_percent', ascending=False).head(3)\n\n# Extracting the names and their ownership percentages\nlargest_shareholders_list = largest_shareholders[['share_holder', 'share_own_percent']]\nlargest_shareholders_list"}`


                                        share_holder  share_own_percent
9                                               Khác             0.2556
0                                    Trương Gia Bình             0.0699
1  Tổng Công ty Đầu Tư Và Kinh Doanh Vốn Nhà Nước...             0.0575Ba cá nhân/công ty là cổ đông lớn nhất và tỷ lệ sở hữu của họ như sau:

1. **Khác**: 25.56%
2. **Trương Gia Bình**: 6.99%
3. **Tổng Công ty Đầu Tư Và Kinh Doanh Vốn Nhà Nước - Công ty TNHH**: 5.75%

> Finished chain.
Tokens Used: 880
	Prompt Tokens: 707
	Completio

### Trading

In [10]:
stock.listing.all_symbols()

,ticker,organ_name
0,A32,Công ty Cổ phần 32
1,AAA,Công ty Cổ phần Nhựa An Phát Xanh
2,AAM,Công ty Cổ Phần Thủy Sản MeKong
3,AAS,Công ty Cổ phần Chứng khoán SmartInvest
4,AAT,Công ty Cổ phần Tập đoàn Tiên Sơn Thanh Hóa
...,...,...
1593,XPH,Công ty Cổ phần Xà phòng Hà Nội
1594,YBC,Công ty Cổ phần Xi măng và Khoáng sản Yên Bái
1595,YBM,Công ty Cổ phần Khoáng sản Công nghiệp Yên Bái
1596,YEG,Công ty Cổ phần Tập đoàn Yeah1


In [12]:
stock.listing.symbols_by_group('HOSE')

0      AAA
1      AAM
2      AAT
3      ABR
4      ABS
      ... 
411    VTB
412    VTO
413    VTP
414    YBM
415    YEG
Name: symbol, Length: 416, dtype: object

In [32]:
df = stock.quote.history(start='2024-06-01', end='2024-08-20', interval='15m')
df

,time,open,high,low,close,volume
0,2024-06-03 09:15:00,24.85,24.95,24.70,24.75,392800
1,2024-06-03 09:30:00,24.75,24.80,24.60,24.65,1016600
2,2024-06-03 09:45:00,24.65,24.70,24.60,24.60,544000
3,2024-06-03 10:00:00,24.60,24.75,24.60,24.70,543600
4,2024-06-03 10:15:00,24.70,24.75,24.65,24.70,497300
...,...,...,...,...,...,...
7506,2024-08-20 13:45:00,24.10,24.25,24.10,24.25,721700
7507,2024-08-20 14:00:00,24.25,24.30,24.15,24.15,496700
7508,2024-08-20 14:15:00,24.15,24.20,24.15,24.15,634500
7509,2024-08-20 14:30:00,NaN,NaN,NaN,NaN,0


In [17]:
# Bước giá và khối lượng giao dịch: realtime
stock.quote.price_depth('ACB')

,price,acc_volume,acc_buy_volume,acc_sell_volume,acc_undefined_volume
0,24300,78000,78000,0,0
1,24250,425700,155000,270700,0
2,24200,426900,340000,86900,0
3,24150,1032200,437200,595000,0
4,24100,1757700,952400,482600,322700
5,24050,1368500,577100,758800,32600
6,24000,324000,21300,302700,0


In [28]:
company = Vnstock().stock(symbol='ACB', source='TCBS').company

In [1]:
company.shareholders()

NameError: name 'company' is not defined